# Server Module

Copyright 2022 Michael George (AKA Logiqx).

This file is part of [sse-results](https://github.com/Logiqx/sse-results) and is distributed under the terms of the GNU General Public License.

sse-results is free software: you can redistribute it and/or modify it under the terms of the GNU General Public License as published by the Free Software Foundation, either version 3 of the License, or (at your option) any later version.

sse-results is distributed in the hope that it will be useful, but WITHOUT ANY WARRANTY; without even the implied warranty of MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE. See the GNU General Public License for more details.

You should have received a copy of the GNU General Public License along with sse-results. If not, see <https://www.gnu.org/licenses/>.

See https://logiqx.github.io/sse-results/tech/motion.html for further details about the thread model.

In [1]:
import os
import time

from datetime import date, datetime, timedelta

import csv
import json

import logging
import threading
import queue

import socket
from urllib.request import urlopen

# Project modules
from common import Printable, projdir
from threadpool import WorkerThread, WorkerThreadPool
from motiongps import Motion, HOST_STATUS_CONNECTED, HOST_STATUS_COMPLETED
from constants import *

## Default Values

Default values, just in case of missing details in the config file

In [2]:
DEFAULT_LOGGING_LEVEL = logging.INFO

## Startup Functions

Loading configuration, initialising logging, etc

In [3]:
def loadConfig():
    '''Read server config from JSON'''

    filename = os.path.join(projdir, CONFIG_DIR, 'server.json')
    with open(filename, 'r', encoding='utf-8') as f:
        jsonTxt = f.read()
        config = json.loads(jsonTxt)

    return config

In [4]:
def initLogging(config):
    """Initialise logging"""
    
    formatString = "%(asctime)s,%(levelname)s,%(message)s"
    dateTimeFormat = "%Y-%m-%d %H:%M:%S"

    try:
        level = config['Logging']['Level']
    except KeyError:
        level = DEFAULT_LOGGING_LEVEL

    # TODO - implement writing to file
    logging.basicConfig(format=formatString, datefmt=dateTimeFormat, level=level)

## Thread Pools

Creation of thread pools and message queues

In [5]:
def createThreadPools(config, motions, stopEvent):
    """Create the thread pools, including message queues"""

    threadPools = []

    # Detection   
    detectionInQueue = queue.Queue()
    detectionOutQueue = queue.Queue()
    detectionThreadPool = DetectionThreadPool(config, motions, stopEvent, detectionInQueue, detectionOutQueue)
    threadPools.append(detectionThreadPool)

    # Inspection    
    inspectionInQueue = detectionOutQueue
    inspectionOutQueue = queue.Queue()  
    inspectionThreadPool = InspectionThreadPool(config, motions, stopEvent, inspectionInQueue, inspectionOutQueue)
    threadPools.append(inspectionThreadPool)

    # Download
    downloadInQueue = inspectionOutQueue
    downloadOutQueue = queue.Queue()
    downloadThreadPool = DownloadThreadPool(config, motions, stopEvent, downloadInQueue, downloadOutQueue)
    threadPools.append(downloadThreadPool)

    # Disconnect
    disconnectionInQueue = queue.Queue()
    disconnectionOutQueue = queue.Queue()
    disconnectionThreadPool = DisconnectionThreadPool(config, motions, stopEvent, disconnectionInQueue, disconnectionOutQueue)
    threadPools.append(disconnectionThreadPool)

    return threadPools

## Motions

Creating of Motion objects in a dictionary

In [6]:
def dateRange(startDate, endDate):
    '''Determine range of dates'''

    dates = []

    for i in range(int((endDate - startDate).days) + 1):
        dates.append(startDate + timedelta(days = i))
        
    return dates

In [7]:
def readMotions(config):
    '''Read Motions from CSV file'''
    
    motions = {}

    year = datetime.now().year
    csvPath = os.path.join(projdir, EVENTS_DIR, str(year), CONFIG_DIR, 'motions.csv')

    with open(csvPath, 'r', encoding='utf-8') as f:
        csvReader = csv.DictReader(f)
        for motionUser in csvReader:
            motionId = int(motionUser['Motion ID'])
            fileId = motionUser['File ID']

            startDate = datetime.strptime(motionUser['Start Date'], '%Y-%m-%d')
            endDate = datetime.strptime(motionUser['End Date'], '%Y-%m-%d')
            
            if motionId not in motions:
                motion = Motion(config, motionId, fileId=fileId)              
                motions[motionId] = motion
            else:
                motion = motions[motionId]

            for sessionDate in dateRange(startDate, endDate):
                sessionDate = sessionDate.strftime('%Y-%m-%d')

                # Note: Motion ID of zero (or negative) is a dummy record in the config
                if motionId > 0:
                    if sessionDate in motion.users:
                        logging.warning("Motion %i is assigned to multiple people on %s", motionId, sessionDate)
                    else:
                        motion.users[sessionDate] = motionUser
     
    return motions

## Detection Thread Pool

Detect new motions appearing on the network

In [8]:
class DetectionThread(WorkerThread):
    """Thread pool worker to detect a motion appearing on the network"""

    def process(self, motion):
        """Do the actual processing of a single item"""

        if motion.status <= 0:
            logging.debug("%s is processing %s", self.name, motion.hostname)
        
            addrInfo = motion.getAddrInfo()
            
            if addrInfo:
                logging.debug("%s detected at %s", motion.hostname, addrInfo[0])

                status = motion.getStatus(timeout=self.timeout)
                
                if status == HOST_STATUS_CONNECTED:
                    logging.info("%s detected and is accepting connections", motion.hostname)
        
                    self.outQueue.put(motion)

                else:
                    logging.debug("%s detected but is not accepting connections - status %i", motion.hostname, status)
            
            else:
                logging.debug("%s not detected (address information unavailable)", motion.hostname)

In [9]:
class DetectionThreadPool(WorkerThreadPool):
    """Thread pool manager to detect motions appearing on the network"""

    def __init__(self, config, motions, stopEvent, inQueue, outQueue):
        """Simple initialisation"""
        
        super().__init__(config, motions, stopEvent, inQueue, outQueue, DetectionThread)
        

    def process(self):
        """The thread pool manager puts motions onto the in queue for workers to consume"""
        
        logging.debug("%s is queuing motions", self.name)
        
        count = 0
        for motion in self.motions.values():
            if motion.status <= 0:
                logging.debug("%s is queuing %s", self.name, motion.hostname)
                self.inQueue.put(motion)
                count += 1
                
        return count

## Motion Inspection Thread

Inspect new motions appearing on the network

In [10]:
class InspectionThread(WorkerThread):
    """Thread pool worker to inspect a motion appearing on the network"""

    def process(self, motion):
        """Do the actual inspection of a single motion"""
        
        logging.debug("%s is processing %s", self.name, motion.hostname)

        self.motion = motion

        motion.fetchInfo()
        motion.checkInfo()
        
        motion.fetchSettings()
        motion.checkSettings()
        
        motion.fetchLogs()

        # Change to HOST_STATUS_INSPECTED when downloads are implemented
        motion.status = HOST_STATUS_COMPLETED
        
        logging.debug("%s successfully processed %s", self.name, motion.hostname)

        #self.outQueue.put(motion)

In [11]:
class InspectionThreadPool(WorkerThreadPool):
    """Thread pool manager to inspect motions appearing on the network"""

    def __init__(self, config, motions, stopEvent, inQueue, outQueue):
        """Simple initialisation"""
        
        super().__init__(config, motions, stopEvent, inQueue, outQueue, InspectionThread)


    def main(self):
        """There is no processing to be done in an unmanaged pool"""

        pass

## Download Thread Pool

Download new tracks for Motions

In [12]:
class DownloadThread(WorkerThread):
    """Thread pool worker to download tracks from a motion"""

    def process(self, motion):
        """Do the actual downloads for a single motion"""
        
        logging.debug("%s is processing %s", self.name, motion.hostname)
        
        self.outQueue.put(motion)

In [13]:
class DownloadThreadPool(WorkerThreadPool):
    """Thread pool manager to download tracks from motions"""

    def __init__(self, config, motions, stopEvent, inQueue, outQueue):
        """Simple initialisation"""
        
        super().__init__(config, motions, stopEvent, inQueue, outQueue, DownloadThread)


    def main(self):
        """There is no processing to be done in an unmanaged pool"""

        pass

## Disconnection Thread Pool

Detect motions disconnecting from the network

In [14]:
class DisconnectionThread(WorkerThread):
    """Thread pool worker to detect a motion disconnected from the network"""

    def process(self, motion):
        """Do the actual processing of a single item"""
        
        logging.debug("%s is processing %s", self.name, motion.hostname)
        
        status = motion.getStatus(timeout=self.timeout)

        if motion.status > 0:
            logging.debug("%s is still connected", motion.hostname)
            
            # Note that motion.getStatus() will have nuked the status, thus need to set back to HOST_STATUS_COMPLETED
            motion.status = HOST_STATUS_COMPLETED

        else:
            logging.info("%s has been disconnected", motion.hostname)

In [15]:
class DisconnectionThreadPool(WorkerThreadPool):
    """Thread pool manager to detect motions disconnected from the network"""

    def __init__(self, config, motions, stopEvent, inQueue, outQueue):
        """Simple initialisation"""
        
        super().__init__(config, motions, stopEvent, inQueue, outQueue, DisconnectionThread)


    def process(self):
        """The thread pool manager puts motions onto the in queue for workers to consume"""
        
        logging.debug("%s queuing motions", self.name)
        
        count = 0
        for motion in self.motions.values():
            if motion.status == HOST_STATUS_COMPLETED:
                logging.debug("%s is queuing %s", self.name, motion.hostname)
                self.inQueue.put(motion)
                count +=1
            
        return count

## Main Function

Startup and shutdown of all threads

In [16]:
def main():
    """Main function to start all of the thread pools"""

    # Read config
    config = loadConfig()

    # Initialise logging module
    initLogging(config)
    
    # Read Motion data
    motions = readMotions(config)

    # Global event so that all threads can exit gracefully
    stopEvent = threading.Event()
    
    # Create the thread pools, including the message queues, etc.
    logging.debug("Creating thread pools")
    threadPools = createThreadPools(config, motions, stopEvent)

    # Start up all of the thread pools
    abort = True
    try:
        logging.info("Starting thread pools")

        for threadPool in threadPools:
            threadPool.start()

        logging.info("Thread pools started")

        abort = False
    
    except Exception:
        logging.error("Thread pools failed to start")

    # Just sleep until a keyboard interrupt occurs
    while not abort:
        try:
            time.sleep(10)
        
        # Intercept keyboard interrupts to ensure that all threads are properly stopped, prior to exit
        except KeyboardInterrupt:
            logging.info("Keyboard interrupt")
            abort = True

    # Stop threads gracefully
    logging.info("Stopping thread pools")
    stopEvent.set()
    for threadPool in threadPools:
        threadPool.join()
    logging.info("Thread pools stopped")


if __name__ == '__main__':
    main()

2022-10-08 14:56:05,INFO,Starting thread pools
2022-10-08 14:56:05,INFO,Thread pools started
2022-10-08 14:56:05,INFO,motion-611 detected and is accepting connections
2022-10-08 14:56:05,INFO,motion-615 detected and is accepting connections
2022-10-08 14:56:05,INFO,motion-611 battery level is 62%
2022-10-08 14:56:06,INFO,motion-615 battery level is 69%
2022-10-08 14:56:07,INFO,Keyboard interrupt
2022-10-08 14:56:07,INFO,Stopping thread pools
2022-10-08 14:56:08,INFO,Thread pools stopped
